In [1]:
print("hello")

hello


In [15]:
import tensorflow as tf
import torch
import torchvision.transforms as transforms
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.neighbors import NearestNeighbors
from PIL import Image
import os
from tqdm import tqdm
import faiss
import clip
import torch.nn as nn

class ImageSimilaritySearch:
    def __init__(self, dataset_path, method='resnet'):
        self.dataset_path = dataset_path
        self.method = method
        self.image_paths = []
        self.features = None
        self.categories = ['bags', 'dress', 'pants', 'shorts', 'upperwear']
        
        # Initialize the chosen model
        if method == 'resnet':
            self.model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
        elif method == 'efficientnet':
            self.model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
        elif method == 'clip':
            self.model, self.preprocess = clip.load("ViT-B/32", device="mps" if torch.mps.is_available() else "cpu")
            
    def preprocess_image(self, img_path):
        if self.method in ['resnet', 'efficientnet']:
            img = image.load_img(img_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = tf.keras.applications.resnet50.preprocess_input(x)
            return x
        elif self.method == 'clip':
            img = Image.open(img_path)
            return self.preprocess(img).unsqueeze(0)

    def extract_features(self):
        features = []
        self.image_paths = []

        for category in self.categories:
            category_path = os.path.join(self.dataset_path, category)
            if os.path.exists(category_path):
                for img_name in tqdm(os.listdir(category_path)):
                    img_path = os.path.join(category_path, img_name)
                    try:
                        if self.method in ['resnet', 'efficientnet']:
                            img_tensor = self.preprocess_image(img_path)
                            feature = self.model.predict(img_tensor, verbose=0)
                            features.append(feature.flatten())
                        elif self.method == 'clip':
                            img_tensor = self.preprocess_image(img_path).to(
                                "mps" if torch.mps.is_available() else "cpu"
                            )
                            with torch.no_grad():
                                feature = self.model.encode_image(img_tensor)
                                features.append(feature.cpu().numpy().flatten())
                        
                        self.image_paths.append(img_path)
                    except Exception as e:
                        print(f"Error processing {img_path}: {str(e)}")

        self.features = np.array(features)
        print(f"Extracted features shape: {self.features.shape}")
        
        # Initialize FAISS index
        self.init_faiss_index()

    def evaluate_retrieval(self, test_queries, ground_truth):
        precisions = []
        recalls = []
        
        for query, true_matches in zip(test_queries, ground_truth):
            results = self.find_similar_images(query)
            retrieved = set([path for path, _ in results])
            true_set = set(true_matches)
            
            # Calculate precision and recall
            precision = len(retrieved.intersection(true_set)) / len(retrieved)
            recall = len(retrieved.intersection(true_set)) / len(true_set)
            
            precisions.append(precision)
            recalls.append(recall)
        
        return {
            'mean_precision': np.mean(precisions),
            'mean_recall': np.mean(recalls),
            'f1_score': 2 * np.mean(precisions) * np.mean(recalls) / (np.mean(precisions) + np.mean(recalls))
        }    

    def init_faiss_index(self):
        # Normalize features for FAISS
        self.features = self.features.astype('float32')
        self.features_normalized = self.features / np.linalg.norm(self.features, axis=1)[:, np.newaxis]
        
        # Build FAISS index
        self.index = faiss.IndexFlatIP(self.features.shape[1])
        self.index.add(self.features_normalized)

    def find_similar_images(self, query_image_path, k=5):
        # Extract features for query image
        if self.method in ['resnet', 'efficientnet']:
            query_tensor = self.preprocess_image(query_image_path)
            query_features = self.model.predict(query_tensor, verbose=0)
        elif self.method == 'clip':
            query_tensor = self.preprocess_image(query_image_path).to(
                "mps" if torch.mps.is_available() else "cpu"
            )
            with torch.no_grad():
                query_features = self.model.encode_image(query_tensor).cpu().numpy()

        # Normalize query features
        query_features = query_features.astype('float32')
        query_features_normalized = query_features / np.linalg.norm(query_features)

        # Search using FAISS
        D, I = self.index.search(query_features_normalized.reshape(1, -1), k)
        
        similar_images = [self.image_paths[idx] for idx in I[0]]
        similarities = D[0]
        
        return list(zip(similar_images, similarities))

# Siamese Network Implementation
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.feature_extractor = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
        self.feature_extractor = nn.Sequential(*list(self.feature_extractor.children())[:-1])
        self.fc = nn.Linear(512, 128)
        self.image_paths = []
        self.features = None
        self.device = "mps" if torch.mps.is_available() else "cpu"
        self.to(self.device)
        
    def forward_one(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
    def forward(self, x1, x2):
        out1 = self.forward_one(x1)
        out2 = self.forward_one(x2)
        return out1, out2
    
    def preprocess_image(self, img_path):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
        img = Image.open(img_path).convert('RGB')
        return transform(img).unsqueeze(0)
    
    def extract_features(self, dataset_path):
        self.image_paths = []
        features = []
        self.eval()  # Set to evaluation mode
        
        categories = ['bags', 'dress', 'pants', 'shorts', 'upperwear']
        
        with torch.no_grad():
            for category in categories:
                category_path = os.path.join(dataset_path, category)
                if os.path.exists(category_path):
                    for img_name in tqdm(os.listdir(category_path)):
                        try:
                            img_path = os.path.join(category_path, img_name)
                            img_tensor = self.preprocess_image(img_path).to(self.device)
                            # Extract features using forward_one
                            feature = self.forward_one(img_tensor)
                            features.append(feature.cpu().numpy().flatten())
                            self.image_paths.append(img_path)
                        except Exception as e:
                            print(f"Error processing {img_path}: {str(e)}")
        
        self.features = np.array(features)
        print(f"Extracted features shape: {self.features.shape}")
        
        # Initialize FAISS index
        self.init_faiss_index()
    
    def init_faiss_index(self):
        # Normalize features for FAISS
        self.features = self.features.astype('float32')
        self.features_normalized = self.features / np.linalg.norm(self.features, axis=1)[:, np.newaxis]
        
        # Build FAISS index
        self.index = faiss.IndexFlatIP(self.features.shape[1])
        self.index.add(self.features_normalized)
    
    def find_similar_images(self, query_image_path, k=5):
        self.eval()  # Set to evaluation mode
        
        with torch.no_grad():
            # Preprocess query image
            query_tensor = self.preprocess_image(query_image_path).to(self.device)
            query_features = self.forward_one(query_tensor)
            query_features = query_features.cpu().numpy()

        # Normalize query features
        query_features = query_features.astype('float32')
        query_features_normalized = query_features / np.linalg.norm(query_features)

        # Search using FAISS
        D, I = self.index.search(query_features_normalized.reshape(1, -1), k)
        
        similar_images = [self.image_paths[idx] for idx in I[0]]
        similarities = D[0]
        
        return list(zip(similar_images, similarities))
    
    def evaluate_retrieval(self, test_queries, ground_truth):
        precisions = []
        recalls = []
        
        for query, true_matches in zip(test_queries, ground_truth):
            results = self.find_similar_images(query)
            retrieved = set([path for path, _ in results])
            true_set = set(true_matches)
            
            # Calculate precision and recall
            precision = len(retrieved.intersection(true_set)) / len(retrieved) if retrieved else 0
            recall = len(retrieved.intersection(true_set)) / len(true_set) if true_set else 0
            
            precisions.append(precision)
            recalls.append(recall)
        
        mean_precision = np.mean(precisions)
        mean_recall = np.mean(recalls)
        f1_score = 2 * mean_precision * mean_recall / (mean_precision + mean_recall) if (mean_precision + mean_recall) > 0 else 0
        
        return {
            'mean_precision': mean_precision,
            'mean_recall': mean_recall,
            'f1_score': f1_score
        }
    
    def compute_similarity(self, img1_path, img2_path):
        """Compute similarity between two images using the siamese network"""
        self.eval()
        with torch.no_grad():
            img1_tensor = self.preprocess_image(img1_path).to(self.device)
            img2_tensor = self.preprocess_image(img2_path).to(self.device)
            
            # Get embeddings
            emb1, emb2 = self.forward(img1_tensor, img2_tensor)
            
            # Compute cosine similarity
            similarity = torch.nn.functional.cosine_similarity(emb1, emb2)
            
            return similarity.item()
    
    def train_step(self, img1, img2, label, criterion, optimizer):
        """Single training step for siamese network"""
        # Forward pass
        output1, output2 = self.forward(img1, img2)
        
        # Calculate contrastive loss
        loss = criterion(output1, output2, label)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        return loss.item()
    

class AutoencoderSearch(nn.Module):
    def __init__(self, input_size=224*224*3):
        super().__init__()
        self.input_size = input_size
        self.image_paths = []
        self.features = None
        self.device = "mps" if torch.mps.is_available() else "cpu"
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 32)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, input_size),
            nn.Sigmoid()
        )
        self.to(self.device)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded
    
    def preprocess_image(self, img_path):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        img = Image.open(img_path).convert('RGB')
        return transform(img).unsqueeze(0)
    
    def extract_features(self, dataset_path):
        self.image_paths = []
        features = []
        self.eval()
        
        categories = ['bags', 'dress', 'pants', 'shorts', 'upperwear']
        
        with torch.no_grad():
            for category in categories:
                category_path = os.path.join(dataset_path, category)
                if os.path.exists(category_path):
                    for img_name in tqdm(os.listdir(category_path)):
                        try:
                            img_path = os.path.join(category_path, img_name)
                            img_tensor = self.preprocess_image(img_path).to(self.device)
                            img_tensor = img_tensor.view(img_tensor.size(0), -1)
                            encoded, _ = self.forward(img_tensor)
                            features.append(encoded.cpu().numpy().flatten())
                            self.image_paths.append(img_path)
                        except Exception as e:
                            print(f"Error processing {img_path}: {str(e)}")
        
        self.features = np.array(features)
        print(f"Extracted features shape: {self.features.shape}")
        
        # Initialize FAISS index
        self.init_faiss_index()
    
    def init_faiss_index(self):
        self.features = self.features.astype('float32')
        self.features_normalized = self.features / np.linalg.norm(self.features, axis=1)[:, np.newaxis]
        self.index = faiss.IndexFlatIP(self.features.shape[1])
        self.index.add(self.features_normalized)
    
    def find_similar_images(self, query_image_path, k=5):
        self.eval()
        
        with torch.no_grad():
            query_tensor = self.preprocess_image(query_image_path).to(self.device)
            query_tensor = query_tensor.view(query_tensor.size(0), -1)
            query_features, _ = self.forward(query_tensor)
            query_features = query_features.cpu().numpy()
        
        # Normalize query features
        query_features = query_features.astype('float32')
        query_features_normalized = query_features / np.linalg.norm(query_features)
        
        # Search using FAISS
        D, I = self.index.search(query_features_normalized.reshape(1, -1), k)
        
        similar_images = [self.image_paths[idx] for idx in I[0]]
        similarities = D[0]
        
        return list(zip(similar_images, similarities))
    
    def evaluate_retrieval(self, test_queries, ground_truth):
        precisions = []
        recalls = []
        
        for query, true_matches in zip(test_queries, ground_truth):
            results = self.find_similar_images(query)
            retrieved = set([path for path, _ in results])
            true_set = set(true_matches)
            
            precision = len(retrieved.intersection(true_set)) / len(retrieved) if retrieved else 0
            recall = len(retrieved.intersection(true_set)) / len(true_set) if true_set else 0
            
            precisions.append(precision)
            recalls.append(recall)
        
        mean_precision = np.mean(precisions)
        mean_recall = np.mean(recalls)
        f1_score = 2 * mean_precision * mean_recall / (mean_precision + mean_recall) if (mean_precision + mean_recall) > 0 else 0
        
        return {
            'mean_precision': mean_precision,
            'mean_recall': mean_recall,
            'f1_score': f1_score
        }

class LSHSearch:
    def __init__(self, feature_dimension, num_tables=10, num_bits=8):
        self.num_tables = num_tables
        self.num_bits = num_bits
        self.tables = []
        self.random_vectors = []
        self.image_paths = []
        self.features = None
        self.device = "mps" if torch.mps.is_available() else "cpu"
        
        for _ in range(num_tables):
            random_vec = np.random.randn(num_bits, feature_dimension)
            self.random_vectors.append(random_vec)
            self.tables.append([])  # Changed from dict to list
    
    def preprocess_image(self, img_path):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
        img = Image.open(img_path).convert('RGB')
        return transform(img).unsqueeze(0)
    
    def _hash_vector(self, vector, random_vec):
        projections = np.dot(random_vec, vector)
        return ''.join(['1' if p > 0 else '0' for p in projections])
    
    def index(self, features, image_paths):
        self.features = features
        self.image_paths = image_paths
        self.hash_tables = [{} for _ in range(self.num_tables)]
        
        for idx, feature in enumerate(features):
            for i, random_vec in enumerate(self.random_vectors):
                hash_key = self._hash_vector(feature, random_vec)
                if hash_key not in self.hash_tables[i]:
                    self.hash_tables[i][hash_key] = []
                self.hash_tables[i][hash_key].append(idx)
    
    def extract_features(self, dataset_path):
        self.image_paths = []
        features = []
        categories = ['bags', 'dress', 'pants', 'shorts', 'upperwear']
        
        resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
        feature_extractor = nn.Sequential(*list(resnet.children())[:-1]).to(self.device)
        feature_extractor.eval()
        
        with torch.no_grad():
            for category in categories:
                category_path = os.path.join(dataset_path, category)
                if os.path.exists(category_path):
                    for img_name in tqdm(os.listdir(category_path)):
                        try:
                            img_path = os.path.join(category_path, img_name)
                            img_tensor = self.preprocess_image(img_path).to(self.device)
                            feature = feature_extractor(img_tensor)
                            features.append(feature.cpu().numpy().flatten())
                            self.image_paths.append(img_path)
                        except Exception as e:
                            print(f"Error processing {img_path}: {str(e)}")
        
        self.features = np.array(features)
        print(f"Extracted features shape: {self.features.shape}")
        self.index(self.features, self.image_paths)
    
    def find_similar_images(self, query_image_path, k=5):
        # Extract features for query image
        resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
        feature_extractor = nn.Sequential(*list(resnet.children())[:-1]).to(self.device)
        feature_extractor.eval()
        
        with torch.no_grad():
            query_tensor = self.preprocess_image(query_image_path).to(self.device)
            query_features = feature_extractor(query_tensor).cpu().numpy().flatten()
        
        # Find similar images using LSH
        candidate_indices = set()
        for i, random_vec in enumerate(self.random_vectors):
            hash_key = self._hash_vector(query_features, random_vec)
            if hash_key in self.hash_tables[i]:
                candidate_indices.update(self.hash_tables[i][hash_key])
        
        # Calculate actual distances for candidates
        distances = []
        for idx in candidate_indices:
            feature = self.features[idx]
            distance = np.dot(query_features, feature) / (np.linalg.norm(query_features) * np.linalg.norm(feature))
            distances.append((self.image_paths[idx], distance))
        
        # If no candidates found, return k random images
        if not distances:
            random_indices = np.random.choice(len(self.image_paths), k, replace=False)
            distances = [(self.image_paths[idx], 0.0) for idx in random_indices]
        
        # Return top k results
        return sorted(distances, key=lambda x: x[1], reverse=True)[:k]

    def evaluate_retrieval(self, test_queries, ground_truth):
        precisions = []
        recalls = []
        
        for query, true_matches in zip(test_queries, ground_truth):
            results = self.find_similar_images(query)
            retrieved = set([path for path, _ in results])
            true_set = set(true_matches)
            
            # Calculate precision and recall
            precision = len(retrieved.intersection(true_set)) / len(retrieved) if retrieved else 0
            recall = len(retrieved.intersection(true_set)) / len(true_set) if true_set else 0
            
            precisions.append(precision)
            recalls.append(recall)
        
        mean_precision = np.mean(precisions)
        mean_recall = np.mean(recalls)
        f1_score = 2 * mean_precision * mean_recall / (mean_precision + mean_recall) if (mean_precision + mean_recall) > 0 else 0
        
        return {
            'mean_precision': mean_precision,
            'mean_recall': mean_recall,
            'f1_score': f1_score
        }            

class TripletNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = torch.hub.load('pytorch/vision:v0.10.0', 
                                              'resnet18', pretrained=True)
        self.feature_extractor = nn.Sequential(*list(self.feature_extractor.children())[:-1])
        self.fc = nn.Linear(512, 128)
        self.image_paths = []
        self.features = None
        self.device = "mps" if torch.mps.is_available() else "cpu"
        self.to(self.device)
        
    def forward_one(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
    def forward(self, anchor, positive, negative):
        anchor_out = self.forward_one(anchor)
        positive_out = self.forward_one(positive)
        negative_out = self.forward_one(negative)
        return anchor_out, positive_out, negative_out
    
    def preprocess_image(self, img_path):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
        img = Image.open(img_path).convert('RGB')
        return transform(img).unsqueeze(0)
    
    def extract_features(self, dataset_path):
        self.image_paths = []
        features = []
        self.eval()  # Set to evaluation mode
        
        categories = ['bags', 'dress', 'pants', 'shorts', 'upperwear']
        
        with torch.no_grad():
            for category in categories:
                category_path = os.path.join(dataset_path, category)
                if os.path.exists(category_path):
                    for img_name in tqdm(os.listdir(category_path)):
                        try:
                            img_path = os.path.join(category_path, img_name)
                            img_tensor = self.preprocess_image(img_path).to(self.device)
                            feature = self.forward_one(img_tensor)
                            features.append(feature.cpu().numpy().flatten())
                            self.image_paths.append(img_path)
                        except Exception as e:
                            print(f"Error processing {img_path}: {str(e)}")
        
        self.features = np.array(features)
        print(f"Extracted features shape: {self.features.shape}")
        
        # Initialize FAISS index
        self.init_faiss_index()
    
    def init_faiss_index(self):
        # Normalize features for FAISS
        self.features = self.features.astype('float32')
        self.features_normalized = self.features / np.linalg.norm(self.features, axis=1)[:, np.newaxis]
        
        # Build FAISS index
        self.index = faiss.IndexFlatIP(self.features.shape[1])
        self.index.add(self.features_normalized)
    
    def find_similar_images(self, query_image_path, k=5):
        self.eval()  # Set to evaluation mode
        
        with torch.no_grad():
            # Preprocess query image
            query_tensor = self.preprocess_image(query_image_path).to(self.device)
            query_features = self.forward_one(query_tensor)
            query_features = query_features.cpu().numpy()

        # Normalize query features
        query_features = query_features.astype('float32')
        query_features_normalized = query_features / np.linalg.norm(query_features)

        # Search using FAISS
        D, I = self.index.search(query_features_normalized.reshape(1, -1), k)
        
        similar_images = [self.image_paths[idx] for idx in I[0]]
        similarities = D[0]
        
        return list(zip(similar_images, similarities))
    
    def evaluate_retrieval(self, test_queries, ground_truth):
        precisions = []
        recalls = []
        
        for query, true_matches in zip(test_queries, ground_truth):
            results = self.find_similar_images(query)
            retrieved = set([path for path, _ in results])
            true_set = set(true_matches)
            
            # Calculate precision and recall
            precision = len(retrieved.intersection(true_set)) / len(retrieved) if retrieved else 0
            recall = len(retrieved.intersection(true_set)) / len(true_set) if true_set else 0
            
            precisions.append(precision)
            recalls.append(recall)
        
        mean_precision = np.mean(precisions)
        mean_recall = np.mean(recalls)
        f1_score = 2 * mean_precision * mean_recall / (mean_precision + mean_recall) if (mean_precision + mean_recall) > 0 else 0
        
        return {
            'mean_precision': mean_precision,
            'mean_recall': mean_recall,
            'f1_score': f1_score
        }

class HybridSearch:
    def __init__(self, models_list=None, weights=None):
        # Default weights based on typical performance characteristics
        self.default_weights = {
    'clip': 0.25,      # CLIP for semantic understanding
    'resnet': 0.20,    # ResNet for general features
    'efficientnet': 0.15,  # EfficientNet for efficiency
    'autoencoder': 0.15,   # Autoencoder for latent space representation
    'siamese': 0.10,    # Siamese for paired similarity
    'triplet': 0.10,    # Triplet for relative similarity
    'lsh': 0.05        # LSH for fast approximate search
}
        
        self.models = models_list
        if weights:
            self.weights = weights
        else:
            # Assign weights based on which models are actually provided
            total_weight = sum(self.default_weights[model.method] 
                             for model in models_list 
                             if hasattr(model, 'method') and model.method in self.default_weights)
            
            # Normalize weights to sum to 1
            self.weights = [
                self.default_weights[model.method] / total_weight 
                if hasattr(model, 'method') and model.method in self.default_weights 
                else 1.0 / len(models_list) 
                for model in models_list
            ]

    def _combine_results(self, all_results, k):
        combined_scores = {}
        
        # Weighted voting with normalized scores
        for (results, weight) in all_results:
            # Normalize scores within each model's results
            max_score = max(score for _, score in results)
            min_score = min(score for _, score in results)
            score_range = max_score - min_score if max_score != min_score else 1
            
            for path, score in results:
                if path not in combined_scores:
                    combined_scores[path] = 0
                # Normalize score to [0,1] range before applying weight
                normalized_score = (score - min_score) / score_range
                combined_scores[path] += normalized_score * weight
        
        # Sort and return top k results
        sorted_results = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)[:k]
        return sorted_results

    def find_similar_images(self, query_image, k=5):
        all_results = []
        
        for model, weight in zip(self.models, self.weights):
            try:
                results = model.find_similar_images(query_image, k)
                all_results.append((results, weight))
            except Exception as e:
                print(f"Error with model {model.__class__.__name__}: {str(e)}")
                continue
        
        return self._combine_results(all_results, k)

    def evaluate_retrieval(self, test_queries, ground_truth):
        precisions = []
        recalls = []
        
        for query, true_matches in zip(test_queries, ground_truth):
            results = self.find_similar_images(query)
            retrieved = set([path for path, _ in results])
            true_set = set(true_matches)
            
            precision = len(retrieved.intersection(true_set)) / len(retrieved) if retrieved else 0
            recall = len(retrieved.intersection(true_set)) / len(true_set) if true_set else 0
            
            precisions.append(precision)
            recalls.append(recall)
        
        mean_precision = np.mean(precisions)
        mean_recall = np.mean(recalls)
        f1_score = 2 * mean_precision * mean_recall / (mean_precision + mean_recall) if (mean_precision + mean_recall) > 0 else 0  
        return {
            'mean_precision': mean_precision,
            'mean_recall': mean_recall,
            'f1_score': f1_score
        }

In [16]:
import time
import random
from datetime import datetime
import pandas as pd

def test_similarity_search():
    dataset_path = "/Users/sammy/Desktop/Shoppin/sampled_fashion_dataset/"
    results_data = []
    
    # Initialize all models
    print("Initializing models...")
    models = {
        'resnet': ImageSimilaritySearch(dataset_path, method='resnet'),
        'clip': ImageSimilaritySearch(dataset_path, method='clip'),
        'efficientnet': ImageSimilaritySearch(dataset_path, method='efficientnet'),
        'autoencoder': AutoencoderSearch(),
        'siamese': SiameseNetwork(),
        'triplet': TripletNetwork(),
        'lsh': LSHSearch(feature_dimension=512)  # 512 for ResNet18 features
    }
    
    # Extract features for all models
    print("\nExtracting features for all models...")
    for name, model in models.items():
        start_time = time.time()
        if name in ['resnet', 'clip', 'efficientnet']:
            model.extract_features()
        else:
            model.extract_features(dataset_path)  # For other models that need dataset_path
        extraction_time = time.time() - start_time
        print(f"{name.upper()} feature extraction time: {extraction_time:.2f} seconds")
    
    # Create test queries and ground truth
    test_queries = []
    ground_truth = []
    
    # Get test images from each category
    print("\nPreparing test images...")
    for category in models['resnet'].categories:
        category_path = os.path.join(dataset_path, category)
        if os.path.exists(category_path):
            category_images = [os.path.join(category_path, img) 
                             for img in os.listdir(category_path)]
            
            # Select 10 random images for testing from each category
            if len(category_images) > 10:
                test_images = random.sample(category_images, 10)
                for test_img in test_images:
                    test_queries.append(test_img)
                    truth = [img for img in category_images if img != test_img]
                    ground_truth.append(truth)
    
    # Evaluate each model
    print("\nEvaluating all models...")
    for model_name, model in models.items():
        print(f"\nEvaluating {model_name.upper()}:")
        
        # Time the evaluation
        start_time = time.time()
        if hasattr(model, 'evaluate_retrieval'):
            metrics = model.evaluate_retrieval(test_queries, ground_truth)
            eval_time = time.time() - start_time
            
            print(f"Metrics for {model_name}:")
            print(f"Mean Precision: {metrics['mean_precision']:.4f}")
            print(f"Mean Recall: {metrics['mean_recall']:.4f}")
            print(f"F1 Score: {metrics['f1_score']:.4f}")
            print(f"Evaluation Time: {eval_time:.2f} seconds")
            
            # Store results
            results_data.append({
                'model': model_name,
                'precision': metrics['mean_precision'],
                'recall': metrics['mean_recall'],
                'f1_score': metrics['f1_score'],
                'eval_time': eval_time
            })
    
    # Test hybrid approach
    print("\nTesting Hybrid Approach...")
    hybrid_model = HybridSearch([
        models['resnet'], 
        models['clip'],
        models['efficientnet'],
        models['autoencoder'],
        models['siamese'],
        models['triplet'],
        models['lsh']
    ])
    start_time = time.time()
    hybrid_metrics = hybrid_model.evaluate_retrieval(test_queries[:10], ground_truth[:10])
    hybrid_time = time.time() - start_time
    
    results_data.append({
        'model': 'hybrid',
        'precision': hybrid_metrics['mean_precision'],
        'recall': hybrid_metrics['mean_recall'],
        'f1_score': hybrid_metrics['f1_score'],
        'eval_time': hybrid_time
    })
    
    # Create detailed performance report
    results_df = pd.DataFrame(results_data)
    print("\nDetailed Performance Report:")
    print(results_df)
    
    # Save results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_df.to_csv(f'model_comparison_results_{timestamp}.csv', index=False)

    detailed_metrics_data = []
    for idx, row in results_df.iterrows():
        metrics_entry = {
            'Model': row['model'],
            'Mean Precision': f"{row['precision']:.4f}",
            'Mean Recall': f"{row['recall']:.4f}",
            'F1 Score': f"{row['f1_score']:.4f}",
            'Evaluation Time (s)': f"{row['eval_time']:.2f}"
        }
        detailed_metrics_data.append(metrics_entry)
    
    detailed_metrics_df = pd.DataFrame(detailed_metrics_data)

    query_results_data = []
    
    # Test individual queries with detailed logging
    print("\nTesting individual query examples...")
    test_images = random.sample(test_queries, 5)  # Select 5 random test images
    
    for idx, test_image in enumerate(test_images, 1):
        query_category = test_image.split('/')[-2]  # Extract category from path
        
        for model_name, model in models.items():
            if hasattr(model, 'find_similar_images'):
                start_time = time.time()
                results = model.find_similar_images(test_image)
                query_time = time.time() - start_time
                
                # Log each retrieved image
                for rank, (path, similarity) in enumerate(results, 1):
                    retrieved_category = path.split('/')[-2]
                    is_correct = retrieved_category == query_category
                    
                    query_results_data.append({
                        'Query_ID': idx,
                        'Query_Image': test_image,
                        'Query_Category': query_category,
                        'Model': model_name,
                        'Retrieved_Image': path,
                        'Retrieved_Category': retrieved_category,
                        'Rank': rank,
                        'Similarity_Score': similarity,
                        'Is_Correct_Category': is_correct,
                        'Query_Time': query_time
                    })
    
    query_results_df = pd.DataFrame(query_results_data)
    
    # Save both DataFrames
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    detailed_metrics_df.to_csv(f'detailed_metrics_{timestamp}.csv', index=False)
    query_results_df.to_csv(f'query_results_{timestamp}.csv', index=False)
    
    # Print summary tables
    print("\n=== Detailed Model Performance Metrics ===")
    print(detailed_metrics_df.to_string())
    
    print("\n=== Query Results Analysis ===")
    summary = query_results_df.groupby('Model').agg({
        'Is_Correct_Category': 'mean',
        'Query_Time': 'mean',
        'Similarity_Score': ['mean', 'std']
    }).round(4)
    print(summary)
    
    # Create model architecture details
    model_details = {
        'resnet': 'ResNet50 (ImageNet pretrained)',
        'clip': 'CLIP ViT-B/32',
        'efficientnet': 'EfficientNetB0',
        'autoencoder': 'Custom AE (32-dim latent)',
        'siamese': 'Siamese ResNet18',
        'triplet': 'Triplet ResNet18',
        'lsh': 'LSH (512-dim, 10 tables)',
        'hybrid': 'Weighted Ensemble'
    }
    
    feature_dimensions = {
        'resnet': 2048,
        'clip': 512,
        'efficientnet': 1280,
        'autoencoder': 32,
        'siamese': 128,
        'triplet': 128,
        'lsh': 512,
        'hybrid': 'Variable'
    }
    
    # Update detailed metrics with architecture info
    detailed_metrics_df['Model Architecture'] = detailed_metrics_df['Model'].map(model_details)
    detailed_metrics_df['Feature Dimension'] = detailed_metrics_df['Model'].map(feature_dimensions)
    
    return detailed_metrics_df, query_results_df

In [17]:
metrics_df, results_df = test_similarity_search()

Initializing models...


Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0



Extracting features for all models...


100%|██████████| 100/100 [00:09<00:00, 10.07it/s]


Extracted features shape: (500, 2048)
RESNET feature extraction time: 45.69 seconds


...

Metrics for lsh:
Mean Precision: 0.6800
Mean Recall: 0.0343
F1 Score: 0.0654
Evaluation Time: 10.84 seconds

Testing Hybrid Approach...


Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0



Detailed Performance Report:
          model  precision    recall  f1_score  eval_time
0        resnet      0.700  0.035354  0.067308   4.560920
1          clip      0.648  0.032727  0.062308   1.298262
2  efficientnet      0.640  0.032323  0.061538   3.026808
3   autoencoder      0.516  0.026061  0.049615   1.014230
4       siamese      0.656  0.033131  0.063077   0.807374
5       triplet      0.672  0.033939  0.064615   0.768623
6           lsh      0.680  0.034343  0.065385  10.840207
7        hybrid      0.740  0.037374  0.071154   4.586196

Testing individual query examples...


Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/sammy/Desktop/Shoppin/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sammy/Desktop/Shoppin/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/sammy/.


=== Detailed Model Performance Metrics ===
          Model Mean Precision Mean Recall F1 Score Evaluation Time (s) Average Query Time (s) Memory Usage (MB) Model Architecture Feature Dimension
0        resnet         0.7000      0.0354   0.0673                4.56                   None              None               None              None
1          clip         0.6480      0.0327   0.0623                1.30                   None              None               None              None
2  efficientnet         0.6400      0.0323   0.0615                3.03                   None              None               None              None
3   autoencoder         0.5160      0.0261   0.0496                1.01                   None              None               None              None
4       siamese         0.6560      0.0331   0.0631                0.81                   None              None               None              None
5       triplet         0.6720      0.0339   0.0646     